# Examples on using the Basel Precision Instruments LNHR DAC II Telnet module

Copyright (c) Basel Precision Instruments GmbH (2024)

## imports and setup

In [1]:
import telnet

# only used for a custom made waveform
from math import sin
from math import cos

# create an instance of the LNHR DAC device
DAC = telnet.LNHRDAC("192.168.0.5",23)

Connected to DAC (192.168.0.5) successfully. The current status of all channels is shown below (channel 1; channel 2; ... ):
OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF;OFF



## set a DC voltage on a channel

In [2]:
DAC.send_command("all off")

# set DAC channel 1 to 1 volt
DAC.send_command("1 8CCCCC") 
DAC.send_command("1 on")

# set multiple channels (2-6) to -2.5 volts
for channel in range(2, 5):
    DAC.send_command(f"{channel} 600000")
    DAC.send_command(f"{channel} on")

KeyError: 'Error occured: send_command("all off") could not be processed by LNHR DAC, DAC answered: b\'\''

## create a standard waveform with the integrated standard waveform generator

In [6]:
# create a simple sinewave on channel 13
DAC.send_command("13 off")
DAC.send_command("c awg-c stop")

while DAC.expect_query_answer("c awg-c ava?", "0"): # wait for availability of awg-c 
    pass

DAC.send_command("c awg-c ch 13") # select channel for awg
DAC.send_command("c swg mode 0") # generate new waveform
DAC.send_command("c swg wf 0") # choose waveform (0 = sine)
# further properties of waveform could be specified here, i.e. amplitude, offset or phase
DAC.send_command("c swg wmem 2") # select wave-memory to save the generated waveform into
DAC.send_command("c awg-c cs 0") # set number of cycles (0 = infinite cycles)
DAC.send_command("c swg wfun 0") # generated waveform will be copied to the selected wave-memory
DAC.send_command("c swg apply") # apply all changes to wave-memory now
DAC.send_command("c wav-c write") # write content of wave-memory into awg memory

print("writing to memory ...", end="")
while DAC.expect_query_answer("c wav-c busy?", "1"): # wait until waveform is written into awg memory
    print(".", end="")

DAC.send_command("c awg-c start") # start awg
DAC.send_command("13 on") #turn channel on

writing to memory ...

## create a custom waveform using mathematical functions

In [10]:
# create a custom waveform on channel 17
DAC.send_command("17 off")
DAC.send_command("c awg-d stop")

while DAC.expect_query_answer("c awg-d ava?", "0"): # wait for availability of awg-c 
    pass

DAC.send_command("c awg-d ch 17") # select channel for awg
DAC.send_command("c awg-d cs 0") # set number of cycles (0 = infinite cycles)
DAC.send_command("c wav-d clr") # clear wavememory

print("generating waveform ...", end="")
for x in range(0, 34000):
     y = 3*sin(x/200) + 2*cos(0.03*x) + (8/7)*sin(x/100) + 3.7*cos(x/10000) # generating arbitrary curve
     DAC.send_command(f"wav-a {x:x} {y:.6f}", hold_connection=True) # transmitting datapoints as voltage to DAC
     if x % 100 == 0: print(".", end="")
print("\n")

DAC.send_command("c wav-d write") # write content of wave-memory into awg memory

print("writing to memory ...", end="")
while DAC.expect_query_answer("c wav-c busy?", "1"): # wait until waveform is written into awg memory
    print(".", end="")

DAC.send_command("c awg-cd cp 1000") # set clock period, this might interfere with other awg's
DAC.send_command("c awg-d start") # start awg
DAC.send_command("17 on") #turn channel on

generating waveform .......................................................................................................................................................................................................................................................................................................................................................writing to memory ...